In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
''')

In [ ]:
%%javascript

IPython.OutputArea.auto_scroll_threshold = 500

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };

    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }

        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();

In [ ]:
from ipywidgets import widgets
from ipywidgets import HBox, VBox
from IPython.display import display, clear_output
from ipyfilechooser import FileChooser
from ipywidgets import IntProgress

import json
import os
from psutil import process_iter
from signal import SIGTERM

with widgets.Output():
    %load_ext tensorboard

class model_filter:
    
    def __init__(self, file_path):
        self.json_path = file_path
        self.filter_output = widgets.Output()
        self.read_model_info()
        display(self.filter_output)
        
    def get_tuple(self, input_list):
        return_tuple_list = []
        for i in range(len(input_list)):
            return_tuple_list.append(tuple([input_list[i],i+1]))
        return return_tuple_list
    
    def on_get_models_clicked(self, _):
        self.filter_output.clear_output()
        
        application = self.target_application_list[self.target_application_dropdown.value-1]
        if application == 'All':
            application = self.target_application_list[1:]
        else:
            application = [application]
        device = self.target_device_list[self.target_device_dropdown.value-1]
        if device == 'All':
            device = self.target_device_list[1:]
        else:
            device = [device]
        t_class = self.target_class_list[self.target_class_dropdown.value-1]
        if t_class == 'All':
            t_class = self.target_class_list[1:]
        else:
            t_class = [t_class]
        input_dim = self.input_dim_list[self.input_dim_dropdown.value-1]
        if input_dim == 'All':
            input_dim = self.input_dim_list[1:]
        else:
            input_dim = [input_dim]
        network = self.target_network_list[self.target_network_dropdown.value-1]
        if network == 'All':
            network = self.target_network_list[1:]
        else:
            network = [network]
        framework = self.target_framework_list[self.target_framework_dropdown.value-1]
        if framework == 'All':
            framework = self.target_framework_list[1:]
        else:
            framework = [framework]
        
        
        model_ids = self.filter_models(application,device,t_class,input_dim,network,framework)
        json_for_model = []
        with self.filter_output:
            if len(model_ids) == 0:
                print("\033[1m Can not find any model as selected configuration ! \033[0m")
                return
            for i in model_ids:
                json_for_model.append(self.json_data[i+2])
            GUI = dynamic_gui(json_for_model)
            GUI.show()
        
    def filter_models(self, application, device, tclass, dim, network, freamework):
        models = self.model_info_dict
        selected_model = []
        for i in range(len(models)):
            if models[i]['target_device'] not in device:
                continue
            elif models[i]['target_application'] not in application:
                continue
            elif models[i]['target_class'] not in tclass:
                continue
            elif models[i]['input_dim'] not in dim:
                continue
            elif models[i]['target_network'] not in network:
                continue
            elif models[i]['target_framework'] not in freamework:
                continue
            else:
                selected_model.append(i)
        return selected_model
    
    def read_model_info(self):
        style = {'description_width': '120px'}
        self.json_data = json.load(open(self.json_path))
        self.model_info_dict = []
        
        self.target_application_list = ['All']
        self.target_device_list = ['All']
        self.target_class_list = ['All']
        self.input_dim_list = ['All']
        self.target_network_list = ['All']
        self.target_framework_list = ['All']
        
        for i in self.json_data[2:]:
            self.model_info_dict.append(i['model_spec'])
        
        for info in self.model_info_dict:
            if info['target_device'] not in self.target_device_list:
                self.target_device_list.append(info['target_device'])
            if info['target_application'] not in self.target_application_list:
                self.target_application_list.append(info['target_application'])
            if info['target_class'] not in self.target_class_list:
                self.target_class_list.append(info['target_class'])
            if info['input_dim'] not in self.input_dim_list:
                self.input_dim_list.append(info['input_dim'])
            if info['target_network'] not in self.target_network_list:
                self.target_network_list.append(info['target_network'])
            if info['target_framework'] not in self.target_framework_list:
                self.target_framework_list.append(info['target_framework'])
        
        self.target_device_dropdown = widgets.Dropdown(
            options=self.get_tuple(self.target_device_list),
            value = 1,
            description='Target Device:',
            disabled=False,
            style=style,
        )
        self.target_application_dropdown = widgets.Dropdown(
            options=self.get_tuple(self.target_application_list),
            value = 1,
            description='Target Application:',
            disabled=False,
            style=style,
        )
        self.target_class_dropdown = widgets.Dropdown(
            options=self.get_tuple(self.target_class_list),
            value = 1,
            description='Target Class:',
            disabled=False,
            style=style,
        )
        self.input_dim_dropdown = widgets.Dropdown(
            options=self.get_tuple(self.input_dim_list),
            value = 1,
            description='Input Dim:',
            disabled=False,
            style=style,
        )
        self.target_network_dropdown = widgets.Dropdown(
            options=self.get_tuple(self.target_network_list),
            value = 1,
            description='Target Network:',
            disabled=False,
            style=style,
        )
        self.target_framework_dropdown = widgets.Dropdown(
            options=self.get_tuple(self.target_framework_list),
            value = 1,
            description='Target Framework:',
            disabled=False,
            style=style,
        )
        
        get_models_button = widgets.Button(description="Show Models",
                                               layout=widgets.Layout(width='62.5%', height='30px'),
                                               button_style='info')
        get_models_button.on_click(self.on_get_models_clicked)
        
        display(widgets.VBox([widgets.HBox([self.target_device_dropdown, self.target_application_dropdown]),
                             widgets.HBox([self.target_class_dropdown,self.input_dim_dropdown]),
                             widgets.HBox([self.target_network_dropdown, self.target_framework_dropdown]),
                             get_models_button]))
    
    
class gui_generator:
    
    def __init__(self, dynamic_gui_object):
        self.dynamic_gui = dynamic_gui_object
        self.param_list = {}
        self.elements_list = {}
        self.gui_elements = {}
        self.selector_database = {}
        self.parse_models()
        self.root_path = os.getcwd()
        
    def delete_garbage(self):
        self.param_list = {}
        self.elements_list = {}
        self.gui_elements = {}
        self.selector_database = {}
        
    def parse_models(self):
        self.models_info = self.dynamic_gui.json_data
        self.model_list = []
        self.option_tuple_list = []
        index = 1
        for i in self.models_info:
            self.model_list.append(i['model_name'])
            self.option_tuple_list.append((i['model_name'], index))
            index += 1
        self.model_dropdown = widgets.Dropdown(
            options=self.option_tuple_list,
            value = 1,
            description='Select Model:',
            disabled=False,
        )
    
    def parse_json(self):
        self.sections = self.models_info[self.model_dropdown.value-1]['sections']
        self.section_list = list(self.sections.keys())
        for section in self.sections:
            section_data = self.sections[section]
            if '_comment' in section_data:
                del section_data['_comment']
            self.param_list[section] = section_data
            file_browsers = {}
            input_values = {}
            flag_list = []
            for flag in section_data:
                if flag == 'command':
                    continue
                if section_data[flag]['type'] in ['file', 'dir']:
                    file_browsers[flag]= section_data[flag]
                elif section_data[flag]['type'] == "flag":
                    flag_list.append(flag)
                else:
                    input_values[flag] = section_data[flag]
                    
            self.elements_list[section] = {'file_browsers': file_browsers,
                                          'input_values': input_values,
                                          'flags': flag_list}
            
            self.gui_elements[section] = gui_wrapper(self, self.elements_list[section])
        
class gui_wrapper:
    
    def __init__(self, gui_generator_object, widget_names):
        self.widget_ref_dict =  widget_names
        self.widget_list = []
        self.flag_list = []
        self.style = {'description_width': '130px'}
        self.layout = {'width': '400px'}
        self.gui_generator = gui_generator_object
        self.widgets = self.generate_widgets()
    
    def file_selected(self, file_chooser):
        if file_chooser in self.gui_generator.selector_database:
            for file_ref in self.gui_generator.selector_database[file_chooser]:
                if file_ref[1] == 'dir':
                    file_ref[0].set_path(file_chooser.selected,"")
                elif file_ref[1] == 'file':
                    selected_file = file_chooser.selected.split("/")[-1]
                    file_ref[0].set_path(file_chooser.selected[:-1*len(selected_file)], selected_file)
        
    def generate_widgets(self):
        
        for ref in self.widget_ref_dict['file_browsers']:
            if 'name' in self.widget_ref_dict['file_browsers'][ref]:
                argument_name = self.widget_ref_dict['file_browsers'][ref]['name']
            else:
                argument_name = str(ref).replace('_',' ')
            self.widget_list.append(FileChooser('/lattice/',title='<b>'+argument_name+'</b>',
                                                show_hidden=False))
            if 'default' in self.widget_ref_dict['file_browsers'][ref]:
                if self.widget_ref_dict['file_browsers'][ref]['type'] == 'file':
                    self.widget_list[-1].set_path(self.gui_generator.dynamic_gui.model_path,
                                              self.widget_ref_dict['file_browsers'][ref]['default'])
                elif self.widget_ref_dict['file_browsers'][ref]['type'] == 'dir':
                    self.widget_list[-1].set_path(self.widget_ref_dict['file_browsers'][ref]['default'],
                                                 "")
            if 'selector' in self.widget_ref_dict['file_browsers'][ref]:
                file_browser_ref = self.widget_ref_dict['file_browsers'][ref]['selector']
                target_sections = file_browser_ref.split(".")
                if len(target_sections) != 2:
                    continue
                else:
                    if target_sections[0] not in self.gui_generator.elements_list:
                        continue
                    target_file_browsers = self.gui_generator.elements_list[target_sections[0]]['file_browsers']
                    index=list(target_file_browsers.keys()).index(target_sections[1])
                    ref_name=list(target_file_browsers.keys())[index]
                    target_widget = self.gui_generator.gui_elements[target_sections[0]].widget_list[index]
                    if target_widget in self.gui_generator.selector_database:
                        self.gui_generator.selector_database[target_widget].append([self.widget_list[-1],
                            self.gui_generator.elements_list[target_sections[0]]['file_browsers'][ref_name]['type']])

                    else:
                        self.gui_generator.selector_database[target_widget] = []
                        self.gui_generator.selector_database[target_widget].append([self.widget_list[-1],
                            self.gui_generator.elements_list[target_sections[0]]['file_browsers'][ref_name]['type']])
                    target_widget.register_callback(self.file_selected)
            
        for ref in self.widget_ref_dict['input_values']:
            if 'name' in self.widget_ref_dict['input_values'][ref]:
                argument_name = self.widget_ref_dict['input_values'][ref]['name']
            else:
                argument_name = ref.replace('_',' ')
            self.widget_list.append(widgets.Text(description = argument_name,
                                                 value = self.widget_ref_dict['input_values'][ref]['default'],
                                                 disabled=False, style=self.style,
                                                 layout=self.layout))
        for flag in self.widget_ref_dict['flags']:
            if self.flag_list == []:
                self.flag_list.append(widgets.Label(value = r'\({Optional Flags}\)'))
            self.flag_list.append(widgets.Checkbox(description=flag,value=True,disabled=False,indent=False))
        
        return VBox([widgets.VBox(self.widget_list),widgets.VBox(self.flag_list)])
    
    def get_elements(self):
        return VBox([widgets.VBox(self.widget_list),widgets.VBox(self.flag_list)])
        
    def get_value(self, element_name_ref):
        if element_name_ref in self.widget_ref_dict:
            return self.widget_list[list(self.widget_ref_dict.keys()).index(element_name_ref)].value
        else:
            print("Something went wrong !, could not find reference "+str(element_name_ref)+" in GUI elemets")
            
class dynamic_gui:
    def __init__(self, json_data):
    
        self.json_data = json_data
        self.root_dir = os.getcwd()
        
        self.preprocess_scope = widgets.Output()
        self.scope_0 = widgets.Output()
        self.scope_1 = widgets.Output()
        self.scope_2 = widgets.Output()
        self.tb_kill_log = widgets.Output()
        self.tb_log_dir = ""
        
        self.log_out_section = []
        self.element_out_section = []
        
        self.style = {'description_width': '130px'}
        self.layout = widgets.Layout(width='42.5%', height='30px')
        
        self.clone_dir_path = "/lattice/model/"
        
        self.configure_button = widgets.Button(description="Configure Model",
                                               layout=self.layout,
                                               button_style='info')
        self.configure_button.on_click(self.on_configure_clicked)
        self.section_buttons = []
        
        self.select_model()
        
    def select_model(self):
        self.clone_button = widgets.Button(description="Clone Model",
                                           layout=widgets.Layout(width='31%', height='30px'),
                                           button_style='success')
        self.clone_dir_browser = FileChooser('/lattice/model/',
                                             title='<b>Clone Path (default: /lattice/model/)</b>',
                                                     show_hidden=False)
        self.clone_dir_browser.set_path(self.clone_dir_path, "")
        self.clone_dir_browser.register_callback(self.change_clone_directory)
        self.clone_button.on_click(self.on_clone_clicked)
        
        self.gui_instance = gui_generator(self)
        
        display(widgets.VBox([self.gui_instance.model_dropdown, self.clone_dir_browser, self.clone_button]))
        
    def change_clone_directory(self, filechooser):
        self.clone_dir_path = filechooser.selected
        
    def clone_repo(self):
        self.progressbar = IntProgress(min=0, max=100, description='Cloning :',bar_style='success',
                                       layout=widgets.Layout(width='30%', height='30px'))
        with self.scope_0:
            display(self.progressbar)
            model_name = str(self.gui_instance.models_info[self.gui_instance.model_dropdown.value -1]['git_url'])
            cmd = 'git clone ' + model_name +' '+ self.model_path
            self.progressbar.value = 30
            with widgets.Output():
                ! {cmd}
            self.progressbar.value = 100
    
    def on_clone_clicked(self, _):
        
        def remove_repo(_):
            self.scope_0.clear_output()
            os.system('rm -r '+self.model_path)
            self.clone_repo()
            self.run_model()
            
        def keep_repo(_):
            self.scope_0.clear_output()
            print('Working with existing repository !')
            self.run_model()
            
        os.chdir(self.gui_instance.root_path)
        self.scope_1.clear_output()
        self.scope_2.clear_output()
        self.preprocess_scope.clear_output()
        self.clear_model()
        if not os.path.exists(self.clone_dir_path):
            with self.scope_1:
                print("Please select valid destination path for model !")
            return
        model_name = self.gui_instance.model_list[self.gui_instance.model_dropdown.value-1].replace(" ", "-")
        self.model_path=self.clone_dir_path+model_name
        with self.scope_0:
            clear_output()
            if os.path.exists(self.model_path):
                print('Direcory already exist !!','Do you want to remove it first ?')
                yes_button = widgets.Button(description="Yes", layout=widgets.Layout(width='15%', height='30px'),
                                            button_style='success')
                no_button = widgets.Button(description="No", layout=widgets.Layout(width='15%', height='30px'),
                                           button_style='warning')
                yes_button.on_click(remove_repo)
                no_button.on_click(keep_repo)
                display(HBox([yes_button, no_button]))
            else:
                self.clone_repo()
                self.run_model()
    
    def clear_output(self):
        self.scope_1.clear_output()
        self.clear_model()
            
    def clear_model(self):
        for output in self.log_out_section:
            output.clear_output()
        for output in self.element_out_section:
            output.clear_output()
            
    def run_model(self):
        self.section_buttons = []
        self.gui_instance.delete_garbage()
        self.gui_instance.parse_json()
        self.clear_output()
        os.chdir(self.model_path)
        self.select_directories()
        
    def get_value(self, dictionary):
        value = dictionary['default']
        data_type = dictionary['type']
        try:
            if data_type == "int":
                return str(int(value))
            elif data_type == "str":
                return str(value)
            elif data_type == "float":
                return str(float(value))
        except:
            return None
        
    def get_key_value(self, key):
        file_list = list(self.file_browsers.keys())
        input_list = list(self.input_values.keys())
        
        if key in file_list:
            index = file_list.index(key)
            return self.gui_instance.gui_elements[self.section_selected].widget_list[index].selected
            
        elif key in input_list:
            index = input_list.index(key)
            index = index + len(file_list)
            return self.gui_instance.gui_elements[self.section_selected].widget_list[index].value       
        
    def generate_commands(self, section):
        self.section_selected = section
        self.section_selected = self.gui_instance.section_list[self.section_selected]
        command = []
        self.file_browsers = self.gui_instance.elements_list[self.section_selected]['file_browsers']
        self.input_values = self.gui_instance.elements_list[self.section_selected]['input_values']

        for flag in self.gui_instance.param_list[self.section_selected]:
            if flag == 'command':
                command.append(self.gui_instance.param_list[self.section_selected][flag]["run_type"])
            
            elif flag == 'execution_script':
                command.append(self.get_key_value(flag))
                
            elif self.gui_instance.param_list[self.section_selected][flag]['type'] in ["file", 'dir']:
                value = self.get_key_value(flag)
                if value is None:
                    if self.gui_instance.param_list[self.section_selected][flag]['required']:
                        print("Directory or file not selected !")
                        return None
                    else:
                        continue
                else:
                    command.append("--"+flag)
                    command.append(value)
            elif self.gui_instance.param_list[self.section_selected][flag]['type'] == "flag":
                flag_index =self.gui_instance.elements_list[self.section_selected]['flags'].index(flag) + 1
                if self.gui_instance.gui_elements[self.section_selected].flag_list[flag_index].value:
                    command.append("--"+flag)
                    command.append(" ")
            else:
                
                if self.get_key_value(flag) == "":
                    if self.gui_instance.param_list[self.section_selected][flag]['required']:
                        return None
                    else:
                        continue
                command.append("--"+flag)
                command.append(self.get_value({'default':self.get_key_value(flag),
                        'type':self.gui_instance.param_list[self.section_selected][flag]['type']}))
                if command[-1] is None:
                    return None
        return command
    
    def select_directories(self):
        self.scope_1.clear_output()
        with self.scope_1:
            display(VBox([self.configure_button]))
            
    def kill_process(self, _):
        print("killing process !", self.tb_port)
        for proc in process_iter():
            for conns in proc.connections(kind='inet'):
                if conns.laddr.port == int(self.tb_port):
                    proc.send_signal(SIGTERM)
        self.run_tensorboard()
    
    def skip_tensorboard(self, _):
        return
        
    def run_tensorboard(self):
        self.tb_kill_log.clear_output()
        command = ""
        self.cmd.remove('tensorboard')
        for i in self.cmd:
            command = command +i + " "
        %tensorboard --host=0.0.0.0 {command}
        os.chdir(self.root_dir)
            
    def on_button_clicked(self, arg):
        os.chdir(self.model_path)
        button_index = self.section_buttons.index(arg)
        
        self.log_out_section[button_index].clear_output()
            
        with self.log_out_section[button_index]:
            command = ""
            is_magic_command = False
            cmd = self.generate_commands(button_index)
            self.cmd = cmd
            if cmd is None:
                print("Please give proper input in parameters !")
                return
            
            for i in cmd:
                command = command +i + " "
            
            if cmd[0] == 'tensorboard':
                self.tb_kill_log.clear_output()
                display(self.tb_kill_log)
                if '--port' in cmd:
                    port_index = cmd.index('--port') + 1
                    self.tb_port = cmd[port_index]
                    process_flag = False
                    for proc in process_iter():
                        for conns in proc.connections(kind='inet'):
                            if conns.laddr.port == int(self.tb_port):
                                process_flag = True
                                break
                    with self.tb_kill_log:
                        if process_flag:
                            if  self.tb_log_dir == self.cmd[self.cmd.index("--logdir")+1]:
                                self.run_tensorboard()
                                return
                            else:
                                self.tb_log_dir = self.cmd[self.cmd.index("--logdir")+1]
                            print("Process is running on port "+ self.tb_port+ " !")
                            print("Please kill the process or run tensorbord on another port.")
                            print("Do you want to kill the process ?")
                            yes_button = widgets.Button(description="Yes", layout=widgets.Layout(width='15%',
                                                height='30px'),button_style='success')
                            no_button = widgets.Button(description="No", layout=widgets.Layout(width='15%',
                                                height='30px'),button_style='warning')
                            yes_button.on_click(self.kill_process)
                            no_button.on_click(self.skip_tensorboard)
                            display(HBox([yes_button, no_button]))
                        else:
                            self.run_tensorboard()
                return
                
            print("Executing command :", command)
            ! {command}
            print("command :", command," execution completed !" )
        os.chdir(self.root_dir)
    
    def run_preprocessing(self, _):
        with self.preprocesing_out:
            ! chmod +x {self.shell_browser.selected}
            ! bash {self.shell_browser.selected}
    
    def show_preprosessing(self, _):
        self.shell_browser = FileChooser('/lattice/',title='<b>'+'Select Script :'+'</b>',show_hidden=False)
        shell_run = widgets.Button(description="Run", layout=widgets.Layout(width='20%', height='30px'),
                                            button_style='success')
        shell_run.on_click(self.run_preprocessing)
        self.preprocesing_out.clear_output()
        if not self.is_script_data_shown:
            with self.preprocesing_out:
                display(VBox([self.shell_browser, shell_run]))
                self.is_script_data_shown = True
        else:
            self.is_script_data_shown = False
    
    def create_output_scopes(self, number_of_sections):
        self.log_out_section = []
        self.element_out_section = []
        for i in range(number_of_sections):
            self.log_out_section.append(widgets.Output())
            self.element_out_section.append(widgets.Output())
                
    def on_configure_clicked(self, _):
        with self.preprocess_scope:
            self.is_script_data_shown = False
            self.preprocesing_out = widgets.Output()
            pre_script = widgets.Button(description="Run Preprocessing Script",
                                        layout=self.layout, button_style='info')
            pre_script.on_click(self.show_preprosessing)
            display(VBox([pre_script, self.preprocesing_out]))
        
        number_of_sections = len(self.gui_instance.section_list)
        self.create_output_scopes(number_of_sections)
        self.scope_2.clear_output()
        self.clear_model()
        with self.scope_2:
            for section in  self.gui_instance.sections:
                with self.element_out_section[self.gui_instance.section_list.index(section)]:
                    print("\033[1m" + str(section)[0].upper() +str(section[1:]).replace('_', " ") + "\033[0m")
                current_index = self.gui_instance.section_list.index(section)
                button_description = 'Run '+section[0].upper()+section[1:]
                self.section_buttons.append(widgets.Button(description=button_description.replace('_',' '),
                                                           layout=self.layout))
                self.section_buttons[-1].style.button_color = 'orange'
                self.section_buttons[-1].on_click(self.on_button_clicked)
                with self.element_out_section[current_index]:
                    display(VBox([self.gui_instance.gui_elements[section].get_elements(),
                                  self.section_buttons[-1]]))
                display(self.element_out_section[current_index])
                display(self.log_out_section[current_index])
            
    
    def show(self):
        display(VBox([self.scope_0, self.scope_1, self.preprocess_scope, self.scope_2]))

In [ ]:
try:
    root_working_dir
except NameError:
    root_working_dir = os.getcwd()

os.chdir(root_working_dir)

try:
    GUI  = model_filter('model_info.json')
except:
    print("There was exception, Something gone wrong !!")

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code.">
</form>''')